# 自动微分基础
关于如何运行本教程的 Jupyter 笔记本，请参见 [index](./index.ipynb)。


本笔记本简要介绍了 Drake 中的自动微分（Automatic Differentiation, AutoDiff）。内容包括：
- 基本的 Eigen/Numpy 运算的自动微分，
- 动力学系统（如状态更新、输出计算等）的自动微分，
- 仿真采样数据的自动微分。

### Eigen/Numpy 运算
在 Drake 中，自动微分基于 Eigen 的 AutoDiffScalar 实现。因此，任何显式的 Eigen（在 Python 中即 Numpy）运算都可以自动求导。我们来看一个简单例子 $\mathbf{y} = \mathbf{a}^{\top} \mathbf{x} = [1,~ 2] \mathbf{x}$。我们希望通过自动微分获得 $\partial \mathbf{y} / \partial \mathbf{x} = \mathbf{a}^{\top}$。实现方法如下：

In [20]:
import numpy as np
from pydrake.autodiffutils import InitializeAutoDiff, ExtractGradient

a = np.array([1, 2]).reshape([2, -1]) # 2x1 数组
x = np.random.rand(2, 1)
print("double 类型数组:\n", x)
x = InitializeAutoDiff(x)
print("转换为 AutoDiffXd 标量类型数组:\n", x)
y = a.T @ x
dydx = ExtractGradient(y)
print("梯度:", ExtractGradient(y))
np.testing.assert_allclose(a.T, dydx)  # 验证 dy/dx = a^T

double 类型数组:
 [[0.83079991]
 [0.33793007]]
转换为 AutoDiffXd 标量类型数组:
 [[<AutoDiffXd 0.8307999112630265 nderiv=2>]
 [<AutoDiffXd 0.3379300656777465 nderiv=2>]]
梯度: [[1. 2.]]


注意，自动微分只需在常规 Numpy 运算基础上增加两步：1）用 `InitializeAutoDiff(x)` 声明需要对其求导的变量，2）最后用 `ExtractGradient(y)` 提取梯度。

我们也可以对多个变量同时求导。例如 $\mathbf{y} = \mathbf{a}_1^{\top} \mathbf{x}_1 + \mathbf{a}_2^{\top} \mathbf{x}_2$，我们希望分别获得 $\partial \mathbf{y} / \partial \mathbf{x}_1$ 和 $\partial \mathbf{y} / \partial \mathbf{x}_2$。实现如下：

In [21]:
import numpy as np
from pydrake.autodiffutils import InitializeAutoDiffTuple, ExtractGradient

a1 = np.array([1, 2]).reshape([2, -1])
x1 = np.random.rand(2, 1)

a2 = np.array([1, 2, 3]).reshape([3, -1])
x2 = np.random.rand(3, 1)

x1, x2 = InitializeAutoDiffTuple(x1, x2)
y = a1.T @ x1+ a2.T @ x2

dydx = ExtractGradient(y)
print("所有梯度:", dydx)
dydx1 = dydx[0][0:2]
dydx2 = dydx[0][2:]
print("自动微分计算的梯度:")
print("a1^T =", dydx1)
np.testing.assert_allclose(a1.T, [dydx1])
print("a2^T =", dydx2)
np.testing.assert_allclose(a2.T, [dydx2])

所有梯度: [[1. 2. 1. 2. 3.]]
自动微分计算的梯度:
a1^T = [1. 2.]
a2^T = [1. 2. 3.]


注意，`ExtractGradient(y)` 会提取 `y` 对所有通过 `InitializeAutodiffTuple` 声明的变量的导数。

## 动力学系统

### Drake 内置的动力学系统

Drake 的大多数内置系统的动力学仅涉及显式的 Eigen 运算，因此它们都可以自动微分。我们来看一个简单的离散时间 [LinearSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_linear_system.html)，其动力学为：
$x_{t+1} = x_t +  2u_t, ~~~y_{t} = 3x_t + 4u_t.$  。  
对于一般的动力学系统，我们经常需要下一个状态对当前状态 $\partial x_{t+1}/ \partial x_t$  和输入 $\partial x_{t+1}/ \partial u_t$  的导数，以及输出对状态 $\partial y_{t}/ \partial x_t$ 和输入 $\partial y_{t}/ \partial u_t$ 的导数。这里我们将展示如何通过自动微分获得这些导数。首先构建该系统：

In [22]:
import numpy as np
from pydrake.systems.primitives import LinearSystem
from pydrake.autodiffutils import InitializeAutoDiffTuple, ExtractGradient

A = np.array([[1]])
B = np.array([[2]])
C = np.array([[3]])
D = np.array([[4]])
timestep = 1  # so that the system is discrete-time
system = LinearSystem(A, B, C, D, timestep)

print("A system using double:", system)

A system using double: <pydrake.systems.primitives.LinearSystem object at 0x7fb3d5c11800>


默认情况下，系统使用 `double` 作为标量类型。我们需要将其转换为 [drake::AutoDiffXd](https://drake.mit.edu/doxygen_cxx/namespacedrake.html#a35725b277b02aeb79f24fd7f724e6dbc) 类型：

In [23]:
system_ad = system.ToAutoDiffXd()
print("The system converted to AutoDiffXd:", system_ad)

The system converted to AutoDiffXd: <pydrake.systems.primitives.LinearSystem_𝓣AutoDiffXd𝓤 object at 0x7fb3d5c117b0>


我们令 $x_t = 1$，$u_t=1$（或任意实数）

In [24]:
context_ad = system_ad.CreateDefaultContext()
x = np.array([1])
u = np.array([1])
x, u = InitializeAutoDiffTuple(x, u)
context_ad.SetDiscreteState(0, x)
system_ad.get_input_port(0).FixValue(context_ad, u)

然后，我们计算下一个状态对当前状态和输入的导数 $\partial x_{t+1}/ \partial x_t=1$ 和 $\partial x_{t+1}/ \partial u_t=2$：

In [25]:
# allocate the state object
x_next_object = system_ad.AllocateContext().get_discrete_state()  
# store value to x_next_object without modifying context
system_ad.CalcForcedDiscreteVariableUpdate(context_ad, x_next_object)  
# to extract numpy array from the state object
x_next = x_next_object.get_vector(0).CopyToVector()  
grad = ExtractGradient(x_next)
dx_next_dx = grad.flatten()[0]
dx_next_du = grad.flatten()[1]

print("Gradients calculated by automatic differentiation:")
print("dx'/dx =", dx_next_dx)
assert dx_next_dx == 1
print("dx'/du =", dx_next_du)
assert dx_next_du == 2

Gradients calculated by automatic differentiation:
dx'/dx = 1.0
dx'/du = 2.0


以及输出对状态和输入的导数 $\partial y_{t}/ \partial x_t=3$ 和 $\partial x_{t}/ \partial u_t=4$：

In [26]:
output_object = system_ad.AllocateOutput()
system_ad.CalcOutput(context_ad, output_object)
output_port_index = system_ad.get_output_port(0).get_index()
output = output_object.get_vector_data(output_port_index).CopyToVector()
grad = ExtractGradient(output)
dy_dx = grad.flatten()[0]
dy_du = grad.flatten()[1]
print("Gradients calculated by automatic differentiation::")
print("dy/dx =", dy_dx)
assert dy_dx == 3
print("dy/du =", dy_du)
assert dy_du == 4

Gradients calculated by automatic differentiation::
dy/dx = 3.0
dy/du = 4.0


### 编写支持自动微分的自定义动力学系统

你可以通过将 [drake::AutoDiffXd](https://drake.mit.edu/doxygen_cxx/namespacedrake.html#a35725b277b02aeb79f24fd7f724e6dbc) 作为模板参数，编写支持自动微分的 [LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)。在 Python 中，可以使用 [TemplateSystem](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html) 工具实现。我们考虑如下简单的离散时间系统：  

$x_{t+1} = x_t +  2u_t, ~~~y_{t} = x^2_t.$

我们将用离散时间 [LinearSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_linear_system.html) 构建状态方程，并用自定义系统将输入平方作为输出。首先定义线性系统   
$x_{t+1} = x_t +  2u_t, ~~~y_{t} = x_t$：

In [27]:
import numpy as np
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.systems.primitives import LinearSystem
from pydrake.systems.framework import (
    BasicVector_,
    DiagramBuilder,
    LeafSystem_,
)
from pydrake.autodiffutils import InitializeAutoDiffTuple, ExtractGradient

A = np.array([[1]])
B = np.array([[2]])
C = np.array([[1]])
D = np.array([[0]])
timestep = 1
linear_system = LinearSystem(A, B, C, D, timestep)

现在，定义模板化的自定义系统：

In [28]:
@TemplateSystem.define("SquareSystem_")
def SquareSystem_(T):
    class Impl(LeafSystem_[T]):
        def _construct(self, dimension: int, converter=None):
            LeafSystem_[T].__init__(self, converter=converter)
            self.dimension = dimension
            self.input_port = self.DeclareVectorInputPort(
                "input", BasicVector_[T](dimension)
            )
            self.output_port = self.DeclareVectorOutputPort(
                "output",
                BasicVector_[T](dimension),
                self.calc_output,
            )

        def _construct_copy(self, other, converter=None):
            Impl._construct(self, other.dimension, converter=converter)

        def calc_output(self, context, output):
            input_array = self.input_port.Eval(context)
            # Element-wise squared input as the output y = x * x
            output.set_value(input_array * input_array)

    return Impl

SquareSystem = SquareSystem_[None]  # The default system that uses double as the scalar

与[动力学系统建模](./dynamical_systems.ipynb)章节的主要区别在于这里使用了模板类 `LeafSystem_[T]` 和 `BasicVector_[T]`。默认的 `double` 标量类（`SquareSystem = SquareSystem_[None]`）与不使用模板类的版本几乎一致。但通过模板化，Drake 可以自动将其转换为 [drake::AutoDiffXd](https://drake.mit.edu/doxygen_cxx/namespacedrake.html#a35725b277b02aeb79f24fd7f724e6dbc) 类型以支持自动微分。现在我们构建自定义系统并组合完整的系统图：

In [29]:
squared_output = SquareSystem(dimension=1)

In [30]:
builder = DiagramBuilder()
builder.AddSystem(linear_system)
builder.AddSystem(squared_output)
builder.Connect(linear_system.get_output_port(0), squared_output.get_input_port(0))
builder.ExportInput(linear_system.get_input_port(), "input")
builder.ExportOutput(squared_output.get_output_port(), "output")
# The full dynamical system we are considering
system = builder.Build()  
print("Default double systems:\n", system.GetSystems())

Default double systems:
 [<pydrake.systems.primitives.LinearSystem object at 0x7fb3d49fe5c0>, <pydrake.systems.scalar_conversion.SquareSystem_𝓣float𝓤 object at 0x7fb3d5c11ee0>]


注意，虽然我们只构建了默认的 `SquareSystem`（使用 `double` 作为标量），但通过 `ToAutoDiffXd()` 可以将其转换为 [AutoDiffXd](https://drake.mit.edu/doxygen_cxx/namespacedrake.html#a35725b277b02aeb79f24fd7f724e6dbc) 标量类型的系统：

In [31]:
system_ad = system.ToAutoDiffXd()
print("AutoDiffXd systems:\n", system_ad.GetSystems())

AutoDiffXd systems:
 [<pydrake.systems.primitives.LinearSystem_𝓣AutoDiffXd𝓤 object at 0x7fb4501c8c20>, <pydrake.systems.scalar_conversion.SquareSystem_𝓣AutoDiffXd𝓤 object at 0x7fb3d5c69d50>]


现在我们在 $x_t=1$ 和 $u_t=1$ 处计算导数：

In [32]:
context_ad = system_ad.CreateDefaultContext()
x = np.array([1])
u = np.array([1])
x, u = InitializeAutoDiffTuple(x, u)
context_ad.SetDiscreteState(0, x)
system_ad.get_input_port(0).FixValue(context_ad, u)

我们可以得到正确的导数 $\left. \partial y_{t}/ \partial x_t \right|_{x_t=1}= \left. 2 x_t \right|_{x_t=1} = 2$ 和 $\partial y_{t}/ \partial u_t=0$：

In [33]:
output_object = system_ad.AllocateOutput()
system_ad.CalcOutput(context_ad, output_object)
output_port_index = system_ad.GetOutputPort("output").get_index()
output = output_object.get_vector_data(output_port_index).CopyToVector()
grad = ExtractGradient(output)
dy_dx = grad.flatten()[0]
dy_du = grad.flatten()[1]
print("Gradients calculated by automatic differentiation:")
print("dy/dx =", dy_dx)
assert dy_dx == 2
print("dy/du =", dy_du)
assert dy_du == 0

Gradients calculated by automatic differentiation:
dy/dx = 2.0
dy/du = 0.0


## 仿真采样

最后，我们模拟一个连续时间系统，并对仿真采样结果进行自动微分。我们考虑如下线性系统：
$ \dot{x} = -x,~~ y = x.$ 

给定初始状态 $x_0$，其输出解为：
$ y(t) = e^{-t} x_0,$ 

输出对初始状态的导数为：
$ \frac{\partial y}{\partial x_0} = e^{-t},$

该导数仅与时间有关。现在我们将通过自动微分计算该导数，并与解析梯度进行对比：

In [34]:
import numpy as np
def calculate_analytical_gradient(t):
    return np.exp(-t)

我们先构建该线性系统

In [35]:
from pydrake.systems.primitives import LinearSystem, LogVectorOutput
from pydrake.systems.framework import DiagramBuilder
from pydrake.autodiffutils import InitializeAutoDiff, ExtractGradient, AutoDiffXd
from pydrake.systems.analysis import Simulator_

A = np.array([[-1]])
B = np.array([[0]])
C = np.array([[1]])
D = np.array([[0]])
timestep = 0  # so that the system is continuous-time
linear_system = LinearSystem(A, B, C, D, timestep)

builder = DiagramBuilder()
builder.AddSystem(linear_system)
builder.ExportInput(linear_system.get_input_port(), "input")
builder.ExportOutput(linear_system.get_output_port(), "output")
logger = LogVectorOutput(linear_system.get_output_port(), builder, publish_period=0.1)
system = builder.Build()  # The dynamical system we are considering

并将其转换为 `AutoDiffXd` 标量类型

In [36]:
system_ad = system.ToAutoDiffXd()
logger_ad = system_ad.GetSystems()[1]

我们获取 `AutoDiffXd` 版本的 logger，以便后续提取仿真结果。现在我们构建一个使用 `AutoDiffXd` 标量的 `Simulator`，并设置任意初始状态 $x_0 = 5$：

In [37]:
simulator_ad = Simulator_[AutoDiffXd](system_ad)
context_ad = simulator_ad.get_mutable_context()
system_ad.get_input_port(0).FixValue(context_ad, 0)
x0 = np.array([5])
x0 = InitializeAutoDiff(x0)
context_ad.SetContinuousState(x0)
simulator_ad.Initialize()

最后，我们仿真 1 秒，并断言自动微分计算的导数与解析解一致：

In [38]:
simulator_ad.AdvanceTo(1)
log = logger_ad.FindLog(simulator_ad.get_context())
# convert AutoDiffXd back to double
sample_times = np.array([t.value() for t in log.sample_times()])
# calcualte gradients analytically and via autodiff
analytical_gradients = calculate_analytical_gradient(sample_times)
autodiff_gradients = ExtractGradient(log.data()).flatten()
# Let's print the data at an arbitrary sample time
sample_index = 3
print("dy/dx0 at t =", sample_times[sample_index]),
print("Analytical gradient:", analytical_gradients[sample_index])
print("Gradient calculated by autodiff:", autodiff_gradients[sample_index])
# We assert that the autodiff gradients are correct at all sample times
np.testing.assert_allclose(autodiff_gradients, analytical_gradients, atol=1e-6)

dy/dx0 at t = 0.30000000000000004
Analytical gradient: 0.7408182206817179
Gradient calculated by autodiff: 0.7408182212987496


## 延伸阅读

**Drake 中的系统标量类型与转换**  
- [默认标量类型](https://drake.mit.edu/doxygen_cxx/group__default__scalars.html)：介绍 Drake 常用的标量类型，如 `double`、`AutoDiffXd` 和 `symbolic::Expression`。
- [系统标量类型转换](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html)：描述 Drake 系统如何支持标量类型转换，以实现自动微分和符号分析等功能。

**Eigen 自动微分**  
- [Eigen 自动微分简介（PDF）](https://github.com/edrumwri/drake/blob/bbc944fec87f7dac13169c65c961db29906435fb/drake/doc/autodiff_intro/autodiff.pdf)

**Drake Hydroelastic 接触模型的自动微分**  
- *Kurtz, V., & Lin, H.* (2022). Contact-Implicit Trajectory Optimization with Hydroelastic Contact and iLQR. *IEEE/RSJ IROS 2022*. [论文链接](https://ieeexplore.ieee.org/abstract/document/9981686) [代码](https://github.com/vincekurtz/drake_ddp)